In [0]:
import tensorflow as tf

In [0]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive/"

'Ankush Boarding Pass.pdf'
'Ankush_Hore (1).pdf'
'Ankush_Hore (2).pdf'
 Ankush_Hore.pdf
 ASRM_552
 AV_Safety.ipynb
 Books
'CFA Program Curriculum 2017 Level 1 Volume 1-6..pdf'
'Colab Notebooks'
'College list - CS.xlsx'
'DESKTOP BACKUP'
 drive-download-20181231T075723Z-001
 DSC01682.JPG.gdoc
 Fl_Booking.GOFLDANDd1bcd1449157132_6E434_AF1D2R
 Frankl
 Images
 IMG_20181010_223213.jpg
 IMG_20181024_002029.jpg
'[Joseph_Gallian]_Contemporary_Abstract_Algebra_9e_(BookZZ.org).pdf.zip'
'Machine Learning_ A Probabilistic Perspective [Murphy 2012-08-24].pdf'
'McGrawHill - Machine Learning -Tom Mitchell.pdf'
 PANO_20180504_160844.jpg
 pb5.jpg
 pf1.csv
 STAT542
 structures.csv
'TCS_Resume Format_2012 - Copy.doc'
'TCS_Resume Format_2012 - Copy.doc.gdoc'
 test.csv
 train.csv
 WIN_20180202_10_42_13_Pro.mp4


In [0]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import random

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 1.4MB/s 


In [0]:
link_train = "https://drive.google.com/open?id=1-0pw1faPNc0Ci0Lkhc5k1-yGG7tcq0Nv"

In [0]:
fluff, id = link_train.split('=')
print (id) # Verify that you have everything after '='

1-0pw1faPNc0Ci0Lkhc5k1-yGG7tcq0Nv


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')  
train = pd.read_csv('train.csv')

W0809 02:11:42.353917 140262493333376 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

In [0]:
link_st = "https://drive.google.com/open?id=1vw0iYk8oF6uXoFfyNutlQfLPZQbreVIj"

In [0]:
fluff2, id2 = link_st.split('=')
print (id2) # Verify that you have everything after '='

1vw0iYk8oF6uXoFfyNutlQfLPZQbreVIj


In [0]:
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('structures.csv')  
structures = pd.read_csv('structures.csv')

In [0]:
inter_train = train.merge(structures, how='inner', left_on=['molecule_name','atom_index_0'], 
                right_on=['molecule_name','atom_index'])

In [0]:
loaded_train = inter_train.merge(structures, how='inner', left_on=['molecule_name','atom_index_1'], 
                right_on=['molecule_name','atom_index'])

In [0]:
loaded_train_dropped_cols = loaded_train.drop(['atom_index_x','atom_index_y'], axis=1)

In [0]:
type_field = loaded_train_dropped_cols["type"]

lb = LabelBinarizer()

type_field_bin = lb.fit_transform(type_field)

coordinates_df = loaded_train_dropped_cols[["x_x", "y_x", "z_x", "x_y", "y_y", "z_y", "scalar_coupling_constant"]]

coordinates_array = coordinates_df.as_matrix()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [0]:
final_array = np.concatenate((type_field_bin, coordinates_array), axis = 1)

In [0]:
molecule_id = np.zeros(len(final_array))
prev_molecule = "garbage"
i = 0
j = 0
for each_row in loaded_train_dropped_cols.iterrows():
    new_molecule = each_row[1][1]
    if prev_molecule != new_molecule:
        i = i + 1
    molecule_id[j] = i
    prev_molecule = new_molecule
    j = j + 1

In [0]:
molecule_id_T = molecule_id.reshape((len(molecule_id),1))

final_array_2 = np.concatenate((molecule_id_T, final_array), axis = 1)

In [0]:
final_array_3 = np.zeros((4658147, 17))

final_array_3[:,:15] = final_array_2[:,:15]

for i in range(len(final_array_3)):
    final_array_3[i,15] = np.linalg.norm(final_array_3[i, 9:12] - final_array_3[i, 12:15])

final_array_3[:,16] = final_array_2[:,15]

#final_array_3[:25,:]

In [0]:
final_array_3.shape

(4658147, 17)

In [0]:
#old|def convert_to_superarray(final_array_2):
    superarray = np.zeros((85003, 135, 17, 1))
    begin = 0
    end = 0
    mole = 0
    last_flag = 0
    while ((mole <= 85002) and (begin <= 4658146) and (end <= 4658146)):
        while final_array_2[end][0] <= final_array_2[begin][0]:
            if end == 4658146:
                last_flag = 1
                break
            else:
                end = end + 1
        if last_flag == 1:
            end = end + 1
        length = end - begin
        X_batch_temp_t = final_array_2[begin:end, :].copy()
        up = (135 - length) // 2
        if (135 - length) % 2 == 1:
            down = ((135 - length) // 2) + 1
        else:
            down = ((135 - length) // 2)
        X_batch_temp_2 = np.pad(X_batch_temp_t, ((up, down), (0, 0)), 'constant')
        X_batch_temp_3 = X_batch_temp_2.reshape((135, 17, 1))
        superarray[mole] = X_batch_temp_3
        mole = mole + 1
        begin = end
    return superarray

In [0]:
def convert_to_superarray(final_array_2):
    superarray = np.zeros((85003, 135, 17, 1))
    begin = 0
    end = 0
    mole = 0
    last_flag = 0
    while ((mole <= 85002) and (begin <= 4658146) and (end <= 4658146)):
        while final_array_2[end][0] <= final_array_2[begin][0]:
            if end == 4658146:
                last_flag = 1
                break
            else:
                end = end + 1
        if last_flag == 1:
            end = end + 1
        length = end - begin
        X_batch_temp_t = final_array_2[begin:end, :].copy()
        down = 135 - length
        X_batch_temp_2 = np.pad(X_batch_temp_t, ((0, down), (0, 0)), 'constant')
        X_batch_temp_3 = X_batch_temp_2.reshape((135, 17, 1))
        superarray[mole] = X_batch_temp_3
        mole = mole + 1
        begin = end
    return superarray

In [0]:
superarray = convert_to_superarray(final_array_3)

print(superarray.shape)

superarray2 = superarray.copy()
np.random.shuffle(superarray2)

superarray_train = superarray2[:65000, :, :, :].copy()
superarray_val = superarray2[65000:, :, :, :].copy()

print(superarray_train.shape)

X_train = superarray_train[:, :, 1:16, :].copy()
y_train_temp = superarray_train[:, :, 16, :].copy()
y_train = y_train_temp.reshape((65000, 135))

X_val = superarray_val[:, :, 1:16, :].copy()
y_val_temp = superarray_val[:, :, 16, :].copy()
y_val = y_val_temp.reshape((20003, 135))

(85003, 135, 17, 1)
(65000, 135, 17, 1)


In [0]:
X = tf.placeholder(tf.float32, shape=[None, 135, 15, 1])
y = tf.placeholder(tf.float32, shape=[None, 135])
training_flag = 1
batch_size = tf.placeholder(tf.int64)
dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size).repeat()
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()
#filter1 = #60 6 3 \ 76 9 1 3
#filter2 #25 4 6 \ 36 3 3 6
#filter3 #5 5 10 \ 16 2 6 10

W0809 02:26:59.976041 140262493333376 deprecation.py:323] From <ipython-input-25-e8b2f7d840ad>:6: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


In [0]:
with tf.name_scope("cnn"):
    X_normalized = tf.layers.batch_normalization(features, training = training_flag)
    conv1 = tf.layers.conv2d(X_normalized, 3, [76, 10])
    conv1_normalized = tf.layers.batch_normalization(conv1, training = training_flag)
    conv1_out = tf.nn.relu(conv1_normalized)
    conv2 = tf.layers.conv2d(conv1_out, 6, [36, 3])
    conv2_normalized = tf.layers.batch_normalization(conv2, training = training_flag)
    conv2_out = tf.nn.relu(conv2_normalized)
    conv3 = tf.layers.conv2d(conv2_out, 10, [16, 2])
    conv3_normalized = tf.layers.batch_normalization(conv3, training = training_flag)
    conv3_out = tf.nn.relu(conv3_normalized)
    flat = tf.layers.flatten(conv3)
    output = tf.layers.dense(flat, 135)

W0806 05:35:52.299783 139722750404480 deprecation.py:323] From <ipython-input-26-8bfac6555419>:2: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0806 05:35:52.385833 139722750404480 deprecation.py:323] From <ipython-input-26-8bfac6555419>:3: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0806 05:35:52.390032 139722750404480 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instru

In [0]:
with tf.name_scope("cnn"):
    X0 = tf.layers.flatten(features)
    X1 = tf.layers.dense(X0, 5000, activation = None)
    X2 = tf.layers.batch_normalization(X1, training = training_flag)
    X3 = tf.nn.relu(X2)
    X4 = tf.layers.dropout(X3, rate=0.1)
    X5 = tf.layers.dense(X4, 5000, activation = None)
    X6 = tf.layers.batch_normalization(X5, training = training_flag)
    X7 = tf.nn.relu(X6)
    X8 = tf.layers.dropout(X7, rate=0.1)
    X9 = tf.layers.dense(X8, 2500, activation = None)
    X10 = tf.layers.batch_normalization(X9, training = training_flag)
    X11 = tf.nn.relu(X10)
    X12 = tf.layers.dropout(X11, rate=0.1)
    X13 = tf.layers.dense(X12, 2500, activation = None)
    X14 = tf.layers.batch_normalization(X13, training = training_flag)
    X15 = tf.nn.relu(X14)
    X16 = tf.layers.dropout(X15, rate=0.1)
    X17 = tf.layers.dense(X16, 1200, activation = None)
    X18 = tf.layers.batch_normalization(X17, training = training_flag)
    X19 = tf.nn.relu(X18)
    X20 = tf.layers.dropout(X19, rate=0.1)
    X21 = tf.layers.dense(X20, 600, activation = None)
    X22 = tf.layers.batch_normalization(X21, training = training_flag)
    X23 = tf.nn.relu(X22)
    X24 = tf.layers.dropout(X23, rate=0.1)
    X25 = tf.layers.dense(X24, 300, activation = None)
    X26 = tf.layers.batch_normalization(X25, training = training_flag)
    X27 = tf.nn.relu(X26)
    X28 = tf.layers.dropout(X27, rate=0.1)
    output = tf.layers.dense(X28, 135, activation = None)

W0809 03:06:51.970146 140262493333376 deprecation.py:323] From <ipython-input-49-6e612d8da312>:4: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


In [0]:
with tf.name_scope("loss"):
    l2_loss = tf.losses.mean_squared_error(labels, output)

In [0]:
learning_rate = 0.01
with tf.name_scope("train"):
    training_op = tf.train.AdamOptimizer().minimize(l2_loss)
    #training_op = optimizer.minimize(l2_loss)

In [0]:
with tf.name_scope("estimated_MAE"):
    MAE_hat = tf.math.reduce_mean(tf.metrics.mean_absolute_error(labels, output))
    log_MAE_hat = tf.math.log(MAE_hat)

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
EPOCHS = 2000
BATCH_SIZE = 65000
n_batches = 65000 // BATCH_SIZE

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ X: X_train, y: y_train, batch_size: BATCH_SIZE})
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(n_batches):
            _, loss_value, tr_lmae = sess.run([training_op, l2_loss, log_MAE_hat])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}, Train MAE hat: {:.4f}".format(i, tot_loss / n_batches, tr_lmae))
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ X: X_val, y: y_val, batch_size: X_val.shape[0]})
    t_loss, l_mae = sess.run([l2_loss, log_MAE_hat])
    print('Test Loss: {:.4f}, Test MAE hat: {:.4f}'.format(t_loss, l_mae))
    save_path = saver.save(sess, "./my_model_final5.ckpt")

Training...
Iter: 0, Loss: 597.7324, Train MAE hat: 1.3671
Iter: 1, Loss: 590.1426, Train MAE hat: 2.0576
Iter: 2, Loss: 585.4899, Train MAE hat: 2.0538
Iter: 3, Loss: 581.7490, Train MAE hat: 2.0517
Iter: 4, Loss: 578.4570, Train MAE hat: 2.0499
Iter: 5, Loss: 575.3834, Train MAE hat: 2.0480
Iter: 6, Loss: 572.6252, Train MAE hat: 2.0462
Iter: 7, Loss: 570.0953, Train MAE hat: 2.0446
Iter: 8, Loss: 567.7588, Train MAE hat: 2.0430
Iter: 9, Loss: 565.4351, Train MAE hat: 2.0416
Iter: 10, Loss: 563.1976, Train MAE hat: 2.0404
Iter: 11, Loss: 560.8977, Train MAE hat: 2.0392
Iter: 12, Loss: 558.6147, Train MAE hat: 2.0381
Iter: 13, Loss: 556.3582, Train MAE hat: 2.0372
Iter: 14, Loss: 554.1417, Train MAE hat: 2.0363
Iter: 15, Loss: 552.0281, Train MAE hat: 2.0356
Iter: 16, Loss: 549.8821, Train MAE hat: 2.0349
Iter: 17, Loss: 547.8173, Train MAE hat: 2.0344
Iter: 18, Loss: 545.7922, Train MAE hat: 2.0339
Iter: 19, Loss: 543.8729, Train MAE hat: 2.0336
Iter: 20, Loss: 541.9485, Train MAE ha

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # Restore variables from disk.
    saver.restore(sess, "./my_model_final3.ckpt")
    print("Model restored.")
    # Check the values of the variables
    sess.run(iter.initializer, feed_dict={ X: X_val, y: y_val, batch_size: X_val.shape[0]})
    out, t_loss, l_mae = sess.run([output, l2_loss, log_MAE_hat])
    print('Test Loss: {:.4f}, Test MAE hat: {:.4f}'.format(t_loss, l_mae))

Model restored.
Test Loss: 18.2971, Test MAE hat: -0.0453


In [0]:
def actual_mae(X_val, y_val, out):
    abs_err = np.absolute(y_val - out)
    summe = np.zeros(8)
    for i in range(8):
        new_X_val = X_val[:,:,i,0]
        mult = abs_err * new_X_val
        summe[i] = np.sum(mult)/np.sum(new_X_val)
    summe = np.log(summe)
    MAE = np.sum(summe)/8.0
    return MAE

In [0]:
mae = actual_mae(X_val, y_val, out)
print(mae)

1.3304788416424806


In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # Restore variables from disk.
    saver.restore(sess, "./my_model_final5.ckpt")
    print("Model restored.")
    # Check the values of the variables
    sess.run(iter.initializer, feed_dict={ X: X_train, y: y_train, batch_size: BATCH_SIZE})
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(n_batches):
            _, loss_value, tr_lmae = sess.run([training_op, l2_loss, log_MAE_hat])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}, Train MAE hat: {:.4f}".format(i, tot_loss / n_batches, tr_lmae))
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ X: X_val, y: y_val, batch_size: X_val.shape[0]})
    t_loss, l_mae = sess.run([l2_loss, log_MAE_hat])
    print('Test Loss: {:.4f}, Test MAE hat: {:.4f}'.format(t_loss, l_mae))
    save_path = saver.save(sess, "./my_model_final6.ckpt")

W0809 03:37:40.269182 140262493333376 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Model restored.
Training...
Iter: 0, Loss: 326.8297, Train MAE hat: 1.3418
Iter: 1, Loss: 325.8465, Train MAE hat: 2.0357
Iter: 2, Loss: 324.9893, Train MAE hat: 2.0358
Iter: 3, Loss: 324.0090, Train MAE hat: 2.0352
Iter: 4, Loss: 323.0121, Train MAE hat: 2.0348
Iter: 5, Loss: 321.9562, Train MAE hat: 2.0341
Iter: 6, Loss: 321.1499, Train MAE hat: 2.0336
Iter: 7, Loss: 320.1131, Train MAE hat: 2.0330
Iter: 8, Loss: 319.0588, Train MAE hat: 2.0323
Iter: 9, Loss: 318.0389, Train MAE hat: 2.0318
Iter: 10, Loss: 317.1223, Train MAE hat: 2.0312
Iter: 11, Loss: 316.1216, Train MAE hat: 2.0305
Iter: 12, Loss: 315.2513, Train MAE hat: 2.0300
Iter: 13, Loss: 314.1132, Train MAE hat: 2.0294
Iter: 14, Loss: 313.0199, Train MAE hat: 2.0287
Iter: 15, Loss: 312.0131, Train MAE hat: 2.0281
Iter: 16, Loss: 311.0743, Train MAE hat: 2.0275
Iter: 17, Loss: 310.1294, Train MAE hat: 2.0268
Iter: 18, Loss: 308.9428, Train MAE hat: 2.0262
Iter: 19, Loss: 307.8678, Train MAE hat: 2.0255
Iter: 20, Loss: 306.78

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # Restore variables from disk.
    saver.restore(sess, "./my_model_final6.ckpt")
    print("Model restored.")
    # Check the values of the variables
    sess.run(iter.initializer, feed_dict={ X: X_val, y: y_val, batch_size: X_val.shape[0]})
    out, t_loss, l_mae = sess.run([output, l2_loss, log_MAE_hat])
    print('Test Loss: {:.4f}, Test MAE hat: {:.4f}'.format(t_loss, l_mae))

Model restored.
Test Loss: 114.8918, Test MAE hat: 1.1163


In [0]:
def actual_mae(X_val, y_val, out):
    abs_err = np.absolute(y_val - out)
    summe = np.zeros(8)
    for i in range(8):
        new_X_val = X_val[:,:,i,0]
        mult = abs_err * new_X_val
        summe[i] = np.sum(mult)/np.sum(new_X_val)
    summe = np.log(summe)
    MAE = np.sum(summe)/8.0
    return MAE

In [0]:
mae = actual_mae(X_val, y_val, out)
print(mae)

2.3549514063901205
